In [ ]:
import functools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize

In [ ]:
df = pd.read_csv('calibration/depthmap_distance_train.csv').iloc[1:]

def f_unoptimized(a, x):
    if a[0] - x <= 0 or a[1] == 0:
        return None
    return np.sqrt(a[0] - x) / a[1]

def f_total_loss(a):
    guesses = df['distance'].map(lambda x: f_unoptimized(a, x))
    return (guesses - df['depthmap_value']).fillna(0.)

guess_params = (12., 3.)
optimal_params = scipy.optimize.least_squares(f_total_loss, guess_params).x

calibration_info = {'x{}'.format(i): x for i, x in enumerate(optimal_params)}
calibration_info['loss'] = f_total_loss(optimal_params)

f_optimized = lambda x: f_unoptimized(optimal_params, x)
df['guess'] = df['distance'].map(f_optimized)

fig, ax = plt.subplots()
ax = df.plot(ax=ax, kind='line', x='distance', y='depthmap_value')
ax = df.plot(ax=ax, kind='line', x='distance', y='guess')
plt.show()
print('Optimal params: {}'.format(optimal_params))


calibration_info = {'x{}'.format(i): x for i, x in enumerate(optimal_params)}
calibration_info['loss'] = sum(f_total_loss(optimal_params) ** 2)
print(calibration_info)

In [ ]:
def score(args):
    a, b = args
    guess = depth['Dist'].map(lambda x: np.sqrt(a - x) / b)
    error = sum((depth['Depth'] - guess).dropna() ** 2)
    return error

scipy.optimize.minimize(score, (12, 3.5))